# D2.4 AI-based predictive model to improve diagnostic performance of CCTA
This notebook presents code to run the ischaemia diagnosis model prototype developed for Task 2.4 of the COMBINE-CT project.
This code expects the following:
- A folder per case in data
- Each case folder will have three subfolders:
    - `image`: Containing the CT scan in DICOM format
    - `segmentations`: Containing files called `heart_ventricle_right.nii.gz`, `heart_atrium_left.nii.gz`, `heart_atrium_right.nii.gz`, `heart_myocardium.nii.gz`, `heart_ventricle_left.nii.gz`. This segmentations can be obtained using [TotalSegmentator](https://github.com/wasserth/TotalSegmentator).
    - `clinical information`: Containing a file called `info.csv` with the variables *Age*, *MasculineSex*, *Weight(kg)*, *Height(cm)*, *HeartRate*, *SystolicBP*, *DiastolicBP*,*Hypertension*, *CurrentSmoker*, *SmokingHistory*, *Dyslipidemia*, *Diabetes* corresponding to that patient.

After this code runs, the output of the last cell will show the probability of presenting ischaemia of each case.

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
from functions import *
from nn_architecture import *

In [ ]:
clin_vars = ['Age','MasculineSex','Weight(kg)','Height(cm)','HeartRate','SystolicBP','DiastolicBP','Hypertension','CurrentSmoker','SmokingHistory','Dyslipidemia','Diabetes'] 
clin_mean = np.array([65.41843971631205, 0.6477541371158393, 77.18912529550828, 166.37825059101655, 65.63829787234043, 135.19621749408984, 77.06619385342789, 0.5508274231678487, 0.1607565011820331, 0.25059101654846333, 0.5390070921985816, 0.19858156028368795])
clin_std = np.array([11.193435277438095, 0.4776700900884992, 14.925415040410405, 9.126051855620974, 11.359966575724563, 24.05090709858473, 12.28177324620382, 0.5021401253055001, 0.3673062053787602, 0.43335338809529556, 0.5125064330536094, 0.3989322300829867])

In [ ]:
# DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE = "cpu"
model = SimpleMultimodalClassifier(
    num_clinical_features=11,
    num_classes=2).to(DEVICE)
model.load_state_dict(torch.load("multimodal_model_all_data_entrega_octubre_2025.pth"))
model.eval()

In [ ]:
for case in os.listdir("data"):
    case_path = os.path.join("data", case)
    if not os.path.isdir(case_path):
        continue    

    ct_image, (x_spacing, y_spacing, z_spacing),_,_,_ = read_ct(os.path.join(case_path, "image"))

    clin_info = pd.read_csv(os.path.join(case_path, "clinical information", "info.csv"))
    clin_info_standarized = (clin_info.squeeze().values - clin_mean) / clin_std

    ra_mask,_ = read_nib_mask(os.path.join(case_path, "segmentations", 'heart_atrium_right.nii.gz'))
    la_mask,_ = read_nib_mask(os.path.join(case_path, "segmentations", 'heart_atrium_left.nii.gz'))
    lv_mask,_ = read_nib_mask(os.path.join(case_path, "segmentations", 'heart_ventricle_left.nii.gz'))
    rv_mask,_ = read_nib_mask(os.path.join(case_path, "segmentations",'heart_ventricle_right.nii.gz'))
    myo_mask,_ = read_nib_mask(os.path.join(case_path, "segmentations", 'heart_myocardium.nii.gz'))

    x, y, z = np.where(ra_mask | la_mask | lv_mask | rv_mask | myo_mask)
    x_min, x_max = np.min(x)-1, np.max(x) + 1
    y_min, y_max = np.min(y)-1, np.max(y) + 1
    z_min, z_max = np.min(z)-1, np.max(z) + 1    

    ct_cropped = ct_image[x_min:x_max, y_min:y_max, z_min:z_max]
    ct_cropped = ct_cropped.transpose([1,0,2])[:,::-1,::-1]

    ct_final = preprocess(ct_cropped, np.array((x_spacing, y_spacing, z_spacing)))

    with torch.no_grad():
        prediction = model((torch.tensor(ct_final[np.newaxis,...], dtype=torch.float32),
                        torch.tensor(clin_info_standarized[np.newaxis,:-1], dtype=torch.float32)))
        prediction_prob = torch.nn.Softmax(dim=-1)(prediction)
    print(f"Case {case} has {100*prediction_prob[0,1]:.1f}% probability of being ischaemic")